In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# =============================================================================
# Helper functions
# =============================================================================

def initial_condition(x):
    """Initial condition u2(x,0):
         u = 2 for x < 0,
         u = 2 - x for 0 <= x <= 1,
         u = 1 for x > 1.
    """
    u = np.empty_like(x)
    u[x < 0] = 2.0
    mask = (x >= 0) & (x <= 1)
    u[mask] = 2.0 - x[mask]
    u[x > 1] = 1.0
    return u

def f(u):
    """Flux function: f(u)=u^2/2"""
    return 0.5 * u**2

def update_ghost_cells(u, u_left, u_right):
    """Set ghost cells to fixed boundary values."""
    u[0] = u_left
    u[-1] = u_right
    return u

def minmod(a, b):
    """Minmod limiter for two arrays/scalars."""
    result = np.where(np.sign(a)==np.sign(b), np.minimum(np.abs(a), np.abs(b)) * np.sign(a), 0.0)
    return result

# =============================================================================
# LLF flux and step
# =============================================================================

def llf_flux(uL, uR):
    """Local Lax–Friedrichs (Rusanov) flux for Burgers equation."""
    a = np.maximum(np.abs(uL), np.abs(uR))
    return 0.5*(f(uL) + f(uR)) - 0.5 * a * (uR - uL)

def llf_step(u, dt, dx, u_left, u_right):
    """One time step using the LLF method.
       u is the array including ghost cells.
    """
    # Update ghost cells (fixed BC)
    u = update_ghost_cells(u, u_left, u_right)
    # compute flux at interfaces
    flux = np.zeros_like(u)
    # there are interfaces between indices 0,1,..., len(u)-1
    for i in range(len(u)-1):
        flux[i+1] = llf_flux(u[i], u[i+1])
    # update physical cells (indices 1 to -2)
    u_new = u.copy()
    for i in range(1, len(u)-1):
        u_new[i] = u[i] - (dt/dx) * (flux[i+1] - flux[i])
    return u_new

# =============================================================================
# Godunov flux and step
# =============================================================================

def godunov_flux(uL, uR):
    """Godunov flux for Burgers equation.
       For convex flux, one may use:
         if uL <= uR:
             if uL >= 0: flux = f(uL)
             elif uR <= 0: flux = f(uR)
             else: flux = 0
         else: # uL > uR
             if (uL + uR) > 0: flux = f(uL)
             else: flux = f(uR)
    """
    if uL <= uR:
        if uL >= 0:
            return f(uL)
        elif uR <= 0:
            return f(uR)
        else:
            return 0.0
    else:
        if (uL + uR) > 0:
            return f(uL)
        else:
            return f(uR)

def godunov_step(u, dt, dx, u_left, u_right):
    """One time step using the first-order Godunov scheme."""
    u = update_ghost_cells(u, u_left, u_right)
    flux = np.zeros_like(u)
    for i in range(len(u)-1):
        flux[i+1] = godunov_flux(u[i], u[i+1])
    u_new = u.copy()
    for i in range(1, len(u)-1):
        u_new[i] = u[i] - (dt/dx)*(flux[i+1] - flux[i])
    return u_new

# =============================================================================
# Second–order RK2 with minmod–limited reconstruction (LLF flux)
# =============================================================================

def compute_slopes(u, dx):
    """Compute the minmod limited slope for physical cells.
       u is the array including ghost cells.
       Returns slopes for indices 1 to len(u)-2.
    """
    du_minus = (u[1:-1] - u[:-2]) / dx
    du_plus  = (u[2:] - u[1:-1]) / dx
    slopes = minmod(du_minus, du_plus)
    return slopes

def reconstruct(u, slopes, dx):
    """Reconstruct left/right states at interfaces for physical cells.
       u: cell averages (including ghost cells).
       slopes: limited slopes for cells 1 to -2.
       Returns:
         uL: reconstructed state at right interface of cell i (for i=1 to N)
         uR: reconstructed state at left interface of cell i+1 (for i=1 to N)
         (Note: interfaces are at indices 2,..., len(u)-2)
    """
    # For physical cells i=1,...,N (in array indices 1:-1), slopes is of length len(u)-2.
    uL = u[1:-1] + 0.5 * slopes * dx   # right-side value for cell i
    uR = u[1:-1] - 0.5 * slopes * dx   # left-side value for cell i
    return uL, uR

def rk2_step(u, dt, dx, u_left, u_right):
    """One RK2 time step using a minmod–limited reconstruction and LLF flux."""
    # Stage 1: update ghost cells
    u = update_ghost_cells(u, u_left, u_right)
    # Compute slopes for physical cells (indices 1:-1)
    slopes = compute_slopes(u, dx)
    # Reconstruct interface states at interfaces i+1/2 for i=1,...,N-1.
    # We need reconstructed right state of cell i and left state of cell i+1.
    uL_rec = u[1:-2] + 0.5 * slopes[:-1] * dx  # cell i: i=1,...,N-1
    uR_rec = u[2:-1] - 0.5 * slopes[1:] * dx     # cell i+1: i=2,...,N
    # Compute flux at interfaces (for i=2,...,N-1 --> interfaces 3/2,...,(N-1)+1/2)
    flux = np.zeros(len(u))
    for i in range(1, len(u)-2):
        flux[i+1] = llf_flux(uL_rec[i-1], uR_rec[i-1])
    # Compute stage 1 update for physical cells (indices 2:-2 correspond to physical cells 2,...,N-1)
    u_star = u.copy()
    for i in range(2, len(u)-2):
        u_star[i] = u[i] - (dt/dx) * (flux[i+1] - flux[i])
    # Update ghost cells for u_star
    u_star = update_ghost_cells(u_star, u_left, u_right)

    # Stage 2: recompute slopes and fluxes with u_star
    slopes_star = compute_slopes(u_star, dx)
    uL_rec_star = u_star[1:-2] + 0.5 * slopes_star[:-1] * dx
    uR_rec_star = u_star[2:-1] - 0.5 * slopes_star[1:] * dx
    flux_star = np.zeros(len(u_star))
    for i in range(1, len(u_star)-2):
        flux_star[i+1] = llf_flux(uL_rec_star[i-1], uR_rec_star[i-1])
    # Final update: u^{n+1} = 0.5*(u^n + u_star - dt/dx*(flux_star[i+1]-flux_star[i]))
    u_new = u.copy()
    for i in range(2, len(u)-2):
        u_new[i] = 0.5*( u[i] + u_star[i] - (dt/dx)*(flux_star[i+1] - flux_star[i]) )
    return u_new

# =============================================================================
# Exact solution for t>=1
# =============================================================================

def exact_solution(x, t):
    """
    For t>=1, the entropy solution for the decreasing initial data is
    piecewise constant with u=2 for x < x_shock and u=1 for x > x_shock,
    where the shock speed s=1.5 and the shock forms at x=1 when t=1.
    """
    if t < 1:
        # For t<1, the solution is still smooth (not used in our comparison).
        sol = np.zeros_like(x)
        # For x from the left constant region:
        sol[x < 0 + 2*t] = 2.0
        # For x from the right constant region:
        sol[x > 1 + t] = 1.0
        # For the middle (smooth) region:
        mask = (x >= 0 + 2*t) & (x <= 1 + t)
        # Here we solve: x = y + (2-y)*t  =>  y = (x-2t)/(1-t) and u = 2-y.
        y = (x[mask] - 2*t) / (1 - t)
        sol[mask] = 2 - y
        return sol
    else:
        shock_location = 1 + 1.5*(t-1)
        return np.where(x < shock_location, 2.0, 1.0)

# =============================================================================
# Main simulation routine
# =============================================================================

def run_simulation(method, T_final, N, CFL, u_left, u_right):
    """
    Run the simulation until time T_final.
    method: a function that takes (u, dt, dx, u_left, u_right) and returns updated u.
    N: number of physical cells.
    CFL: CFL number.
    u_left, u_right: fixed boundary values.

    Returns:
      x: cell-center positions (physical cells only)
      u_phys: the cell average values for the physical domain at final time.
    """
    x_min, x_max = -1.0, 6.0
    L = x_max - x_min
    dx = L / N
    # Create grid for physical cells (cell centers)
    x_phys = np.linspace(x_min + 0.5*dx, x_max - 0.5*dx, N)
    # Create array including ghost cells: one extra at each end.
    u = np.empty(N+2)
    # Set initial condition for physical cells
    u[1:-1] = initial_condition(x_phys)
    # Set ghost cells (fixed BC)
    u = update_ghost_cells(u, u_left, u_right)

    # Determine time step from CFL condition.
    # For Burgers eq, max speed is taken as max(|u|). We use an estimate of 2.
    max_speed = 2.0
    dt_max = CFL * dx / max_speed
    # We choose number of steps so that T_final is reached exactly.
    nsteps = int(np.ceil(T_final / dt_max))
    dt = T_final / nsteps

    t = 0.0
    for n in range(nsteps):
        u = method(u, dt, dx, u_left, u_right)
        t += dt
    # Return physical cell averages
    return x_phys, u[1:-1]

# =============================================================================
# Run and compare the methods
# =============================================================================

# Parameters
N = 300            # number of physical cells
CFL = 0.5
u_left = 2.0       # left BC (x=-1, u=2)
u_right = 1.0      # right BC (x=6, u=1)

# Times at which to compare
T_vals = [1.0, 2.0]

methods = {
    "LLF": llf_step,
    "Godunov": godunov_step,
    "RK2-minmod": rk2_step
}

results = {method: {} for method in methods}

for T in T_vals:
    print(f"\n--- T = {T} ---")
    # Compute the exact solution on the physical grid:
    x_grid = np.linspace(-1 + 0.5*(7/N), 6 - 0.5*(7/N), N)
    u_exact = exact_solution(x_grid, T)
    for name, method in methods.items():
        x_sol, u_num = run_simulation(method, T, N, CFL, u_left, u_right)
        # Compute relative L1 error
        err = np.sum(np.abs(u_num - u_exact)) * (7/N)
        norm_exact = np.sum(np.abs(u_exact)) * (7/N)
        rel_err = err / norm_exact
        results[name][T] = (x_sol, u_num, rel_err)
        print(f"{name}: Relative L1 error = {rel_err:.3e}")

# =============================================================================
# Plotting results
# =============================================================================

for T in T_vals:
    plt.figure(figsize=(8,5))
    # Compute exact solution on a finer grid for plotting
    x_fine = np.linspace(-1, 6, 1000)
    u_ex = exact_solution(x_fine, T)
    plt.plot(x_fine, u_ex, 'k-', linewidth=2, label="Exact")
    for name in methods:
        x_sol, u_num, rel_err = results[name][T]
        plt.plot(x_sol, u_num, '--', label=f"{name} (err={rel_err:.2e})")
    plt.xlabel("x")
    plt.ylabel("u")
    plt.title(f"Solution at T = {T}")
    plt.legend()
    plt.grid(True)
    plt.show()
